In [128]:
import json
import pandas as pd

In [129]:
mainfile = "result_2023_6_8.json"

In [130]:
maindata = []
with open(mainfile, 'r') as f:
    for entry in f:
        oneEntry = json.loads(entry)
        maindata.append(oneEntry)


Removing any redundancies

In [131]:
filtereddata = []
for d in maindata:
    if d.get('report') != None:
        if d.get('report').get('user_id') != None:
            filtereddata.append(d)

In [132]:
type(maindata)

list

Removing all non-GET request entries

In [133]:
getrequests = []
for d in filtereddata:
    if d.get('report').get('request_method') == 'GET':      
        getrequests.append(d)

In [134]:
print(len(maindata))
print(len(filtereddata))
print(len(getrequests))

688185
533229
300719


In [135]:
import copy

users = {}

emptydata = {
    "lastname":0,
    "firstname":0,
    "preferredname":0,
    "id":0,
    "email":0,
    "phone":0,
}

user_data = {
    "URL":copy.deepcopy(emptydata),
    "body":copy.deepcopy(emptydata)
}


In [136]:
for d in getrequests:
    user_id = d.get('report').get('user_id')
    if user_id not in users:
        users[user_id] = copy.deepcopy(user_data)
        print(user_id)

d1wgiodc53k
2wpdvj73oou
zwq78tx53ib
zoumk7kg3yq
9tgpozqqg7d
s4ni7dbb7y
5tpckk3d14e
grr97iqp57r
kix7pup4zh
699mztw4z75
dknglbh1h3
p0fjtio4mr
80yfsk3rrlc
zu7pjmkeoi
4485kqex0zs
4q68v2cg2pl
l9duvpdz9m8
y9y0arr5kpm
jktgjnvw8r


In [137]:
for d in getrequests:
    user_id = d.get('report').get('user_id')
    url_leak_type = d.get('report').get('url_leak_type')
    body_leak_type = d.get('report').get('body_leak_type')
    
    if url_leak_type != None:
        for e in url_leak_type:
            users[user_id]["URL"][e] = users[user_id]["URL"][e] + 1
    
    if body_leak_type != None:
        for e in body_leak_type:
            users[user_id]["body"][e] = users[user_id]["body"][e] + 1

Just turning the above data into a nice table using tabulate

In [138]:
from tabulate import tabulate

data = []
for u in users:
    entry = []
    entry.append(u)
    for e in users.get(u).get("URL"):
        entry.append(users.get(u).get("URL").get(e))
    
    data.append(entry)

headers = ['USER ID', 'Last Name', 'First Name','Pref. Name', 'ID', 'Email', "Phone #"]
table = tabulate(data, headers=headers, tablefmt='grid')
print(table)

+-------------+-------------+--------------+--------------+------+---------+-----------+
| USER ID     |   Last Name |   First Name |   Pref. Name |   ID |   Email |   Phone # |
+=============+=============+==============+==============+======+=========+===========+
| d1wgiodc53k |          42 |            0 |            0 |    3 |      38 |         0 |
+-------------+-------------+--------------+--------------+------+---------+-----------+
| 2wpdvj73oou |           0 |            0 |            0 |    0 |       0 |         0 |
+-------------+-------------+--------------+--------------+------+---------+-----------+
| zwq78tx53ib |           4 |            0 |            0 |    0 |       4 |         0 |
+-------------+-------------+--------------+--------------+------+---------+-----------+
| zoumk7kg3yq |           0 |            0 |            5 |    0 |       0 |         0 |
+-------------+-------------+--------------+--------------+------+---------+-----------+
| 9tgpozqqg7d |      

Evidently, users "dknglbh1h3" and "zu7pjmkeoi" are outliers. Interestingly, this data doesn't quite line up with some older calculations for the number of leaks of each type, though those calculations probably didn't use the same filtering methodology I did. Everything's on the same order of magnitude, so it doesn't look like we have any problems. I've excluded those two users from the data set for the later analyses.

I'm also only showing data from the url_leak_type field, and not the body_leak_type field, even though I collected data from both. As it turns out, no personal information was found in the body_leak_type field. Possibly just encrypted, not absent, though. See below for the dictionaries.

(For what it's worth, rerunning this notebook from the top shuffles the order the USER IDs are presented in. Not sure why, but something to note down, I suppose.)

In [139]:
for u in users:
    print(u + "\t" + str(users.get(u).get("body")) )

d1wgiodc53k	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
2wpdvj73oou	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
zwq78tx53ib	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
zoumk7kg3yq	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
9tgpozqqg7d	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
s4ni7dbb7y	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
5tpckk3d14e	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
grr97iqp57r	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
kix7pup4zh	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
699mztw4z75	{'lastname': 0, 'firstname': 0, 'preferredname': 0, 'id': 0, 'email': 0, 'phone': 0}
dknglbh1h3	{'lastname': 0, 'firs

In [144]:
normalizeddata = []

for d in getrequests: 
    if d.get('report').get('user_id') != "dknglbh1h3" and d.get('report').get('user_id') != "zu7pjmkeoi":
        normalizeddata.append(d)

print(len(normalizeddata))            

186308


In [147]:
leaksOnlyNormalized = []

for d in normalizeddata: 
    if d.get('report').get('initiator_domain') != None:
        leaksOnlyNormalized.append(d)
        
print(len(leaksOnlyNormalized))            

592


The number of entries was reduced by 114,411 via filtering out of the two outlying users. There are 114,021 entries in which dknglbh1h3 or zu7pjmkeoi had a last name leak. Numbers seemingly track-- those two users had their last name "leaked" with nearly every search.

Filtering out entries without an initiator domain, we find that there are only 592 entries (down from the initial >100,000 leaked data entries detected-- so evidently those users were drastically skewing the data.

In [148]:
import copy

sites = {}

leakCount = {
    "lastname":0,
    "firstname":0,
    "preferredname":0,
    "id":0,
    "email":0,
    "phone":0,
}

In [149]:
for d in leaksOnlyNormalized:
    domain = d.get('report').get('initiator_domain')
    if domain not in sites:
        sites[domain] = copy.deepcopy(leakCount)
        print(domain)

https://www.google.com/
https://uchicago.zoom.us/
https://outlook.office.com/
https://www.youtube.com/
https://webshell.suite.office.com/
https://www.loom.com/
https://quizlet.com/
https://calendly.com/
https://scone-pa.clients6.google.com/
https://www.instagram.com/
https://ogs.google.com/
https://docs.google.com/
https://drive.google.com/
https://accounts.google.com/
https://twitter.com/
https://calendar.google.com/
https://mail.google.com/
https://www.facebook.com/
https://www.amazon.com/


The list of initiator domains is worryingly short? Don't know if this pilot dataset is just that small or if something else is going on. Maybe an oversight on my part, but might just also mean we need a bigger dataset to make any more meaningful conclusions.

In [152]:
for d in leaksOnlyNormalized:
    domain = d.get('report').get('initiator_domain')
    url_leak_type = d.get('report').get('url_leak_type')
    
    if url_leak_type != None:
        for e in url_leak_type:
            sites[domain][e] = sites[domain][e] + 1

In [153]:
data = []
for d in sites:
    entry = []
    entry.append(d)
    for e in sites.get(d):
        entry.append(sites.get(d).get(e))
    
    data.append(entry)

headers = ['DOMAIN', 'Last Name', 'First Name','Pref. Name', 'ID', 'Email', "Phone #"]
table = tabulate(data, headers=headers, tablefmt='grid')
print(table)

+---------------------------------------+-------------+--------------+--------------+------+---------+-----------+
| DOMAIN                                |   Last Name |   First Name |   Pref. Name |   ID |   Email |   Phone # |
+=======================================+=============+==============+==============+======+=========+===========+
| https://www.google.com/               |         124 |          388 |           76 |    6 |      96 |        46 |
+---------------------------------------+-------------+--------------+--------------+------+---------+-----------+
| https://uchicago.zoom.us/             |           0 |            2 |            2 |    0 |       0 |         0 |
+---------------------------------------+-------------+--------------+--------------+------+---------+-----------+
| https://outlook.office.com/           |           0 |           20 |           20 |    0 |       8 |         0 |
+---------------------------------------+-------------+--------------+----------

This has already been said, but with internal trackers, I doubt this is the full extent of what data collection is occurring